In [1]:
# import necessary libraries
import gspread
import json
import numpy as np
import pandas as pd
import pygsheets
import os

from fake_en_pubs_dedup import fake_en_pubs_list
from pickle import TUPLE2
from oauth2client.service_account import ServiceAccountCredentials
from formatting_processing import client_email,gs_to_df,upload_data_to_gs,create_google_sheet
import warnings
warnings.filterwarnings("ignore")

# Step1. Data load from GoogleSheet
- Take China team 2023 Tier0&1 Batch 1 as example
- https://docs.google.com/spreadsheets/d/1Z3LeMLYCCkW94k3aN62equv3H8gYevwo1te_Ex9rVMY/edit#gid=1596970043 


In [37]:
# Input & Output Location
# For cloud, add the tageted sheet name and tab name below
sheet_name = "Work Sheet - EN Pub QA - 2023 Tier 0 & Tier 1 Batch 1 "
tab_name = "All EN Pubs final"
# Log in to GCP and create  project then enable Google Drive and Google Sheet API
# Obtain creds - client_email


In [38]:
def gs_to_df(creds_file,sheet_name,tab_title):
    # Read Data from a Spreadsheet
    gc = gspread.service_account(filename=creds_file) # gspread – to interact with Google Spreadsheets
    # extract data from google sheet by the name of the sheet
    sheet = gc.open(sheet_name) 
    #For the first sheet, pass the index 0 and so on.
    sheet_instance = sheet.worksheet(tab_title) 
    # get all the records of the data
    records_data = sheet_instance.get_all_records()
    # convert the json to dataframe
    df = pd.DataFrame(records_data)
    return df, sheet

In [39]:
#load the data
raw_df, raw_df_sheet=gs_to_df('creds.json',sheet_name,tab_name) 
raw_df.shape

(3291, 8)

# Step2. Extract Pubs Containing Non-medical Keywords
- Non-medical Keywords Dictionary: 
- https://docs.google.com/spreadsheets/d/14txDljxsOJFCvLDvSp4IeooZTrxu6aP7vBvUesz5HHc/edit#gid=1722157711 

In [41]:
# Clean-up
# rename
auto_fail_v4 = raw_df 

# Upper letter each columns' title -> External ID    PMID  DOI  Affiliation  Title  Journal  
auto_fail_v4.columns = ['PeopleID', 'Doi', 'PMID', 'language_code', 'Title', 'Journal',
       'UniqueID', 'count']

# exclude [Ariticle in Chinese] publications
auto_fail_v4 = auto_fail_v4[auto_fail_v4.Journal.isin(fake_en_pubs_list) == False]
auto_fail_v4.head()

,PeopleID,Doi,PMID,language_code,Title,Journal,UniqueID,count
0,10540010,10.1007/s00401-017-1669-y,28130638,en,Marginal vitamin A deficiency facilitates Alzh...,Acta neuropathologica,28130638xx10540010,22
1,10540010,10.1152/ajpendo.00215.2017,28851737,en,"Cholesterol metabolism and Cx43, Cx46, and Cx5...",American Journal of Physiology-Endocrinology a...,28851737xx10540010,22
2,10540010,10.1139/bcb-2014-0107,25607236,en,Berberine as a therapy for type 2 diabetes and...,Biochemistry and cell biology = Biochimie et b...,25607236xx10540010,22
3,10540010,10.1186/s12916-017-0800-1,28264723,en,Developmental pathways to adiposity begin befo...,BMC medicine,28264723xx10540010,22
4,10540010,10.1093/brain/aww261,27807026,en,Heterozygous PINK1 p.G411S increases risk of P...,Brain : a journal of neurology,27807026xx10540010,22


In [42]:
# Define function to find non-medical keywords

def Find_Irrelevant_Term(auto_fail_v4):
    # Non-medical Dict Gsheet link - https://docs.google.com/spreadsheets/d/14txDljxsOJFCvLDvSp4IeooZTrxu6aP7vBvUesz5HHc/edit#gid=1722157711
    sheet_name = "Pub Team Resources All in One"
    tab_name = "Irrelevant_Term"
    
    # ========================
    # load the data from google sheets
    Irrelevant_Term, Irrelevant_Term_sheet = gs_to_df(
        'creds.json', sheet_name, tab_name)  # the tab name

    # There're three different tiers in non-medical dict, more explanations see this link - https://docs.google.com/spreadsheets/d/14txDljxsOJFCvLDvSp4IeooZTrxu6aP7vBvUesz5HHc/edit#gid=218936378 
    # To simplify computation, relabeled Tier ['True Irrelevant,'Uncertain','Burden'] to [10000,100,1] for aggregation
    Irrelevant_Term_dictionary = {}
    for i in range(len(Irrelevant_Term)):
        Irrelevant_Term_dictionary[Irrelevant_Term['Term'][i].replace(
            '\n', "").strip()] = Irrelevant_Term['Score'][i]

    # Identify Irrelevant Terms Among selected text columns
    # Create a columns with all useful text info
    auto_fail_v4['all_info'] =  auto_fail_v4['Title'] + " " + auto_fail_v4['Journal'] #+ " " + auto_fail_v4['Affiliation']

    # Get all existed Irrelevant Term
    auto_fail_v4['Irrelevant_Term'] = auto_fail_v4['all_info'].str.findall(
        "|".join(Irrelevant_Term_dictionary.keys())).str.join(",")

    # score them
    auto_fail_v4['Irrelevant_Term_pre_Score'] = auto_fail_v4['all_info'].str.findall(
        "|".join(Irrelevant_Term_dictionary.keys()))  # list format for aggregation

    # set each pub's Index to 0
    auto_fail_v4['Irrelevant_Term_Score'] = 0
    # aggregate each Irrelevant Terms's score
    for i in range(len(auto_fail_v4['Irrelevant_Term_pre_Score'])):
        for k in auto_fail_v4['Irrelevant_Term_pre_Score'][i]:
            auto_fail_v4['Irrelevant_Term_Score'][i] += Irrelevant_Term_dictionary.get(k)


    # Drop Temporary column
    auto_fail_v4.drop(columns=['Irrelevant_Term_pre_Score'], inplace=True)

    return auto_fail_v4


In [43]:
auto_fail_v4= Find_Irrelevant_Term(auto_fail_v4)

In [44]:
# take a look of all pubs contain non-medical keywords
score = auto_fail_v4[auto_fail_v4['Irrelevant_Term_Score'] != 0]
score.head()

,PeopleID,Doi,PMID,language_code,Title,Journal,UniqueID,count,all_info,Irrelevant_Term,Irrelevant_Term_Score
7,10540010,10.1186/s12940-018-0440-8,30606207,en,Air pollution in the week prior to delivery an...,Environmental health : a global access science...,30606207xx10540010,22,Air pollution in the week prior to delivery an...,Environmental,10000
8,10540010,10.1016/j.envres.2016.04.025,27155984,en,A national study of the association between tr...,Environmental research,27155984xx10540010,22,A national study of the association between tr...,Environmental,10000
27,8219880,10.1016/j.bbrc.2013.04.063,23628414,en,Genes involved in the transition from normal e...,Biochemical and Biophysical Research Communica...,23628414xx8219880,57,Genes involved in the transition from normal e...,Communications,10000
120,4670718,10.1038/s41467-018-07022-2,30385786,en,Reciprocal inhibition of YAP/TAZ and NF-κB reg...,Nature Communications,30385786xx4670718,59,Reciprocal inhibition of YAP/TAZ and NF-κB reg...,Nature Communications,10000
215,4694244,10.1037/fam0000233,27513284,en,Happy Family Kitchen: A community-based resear...,Journal of family psychology : JFP : journal o...,27513284xx4694244,141,Happy Family Kitchen: A community-based resear...,Psychology,1


# Step3. Export Containing Non-medical Keywords Pubs

In [45]:
def upload_data_to_gs(title,updated_df,sheet):
    row=updated_df.shape[0]
    col=updated_df.shape[1]
    # add a sheet with 20 rows and 2 columns
    sheet.add_worksheet(rows=row,cols=col,title=title)
    # get the instance of the second sheet
    new_sheet = sheet.worksheet(title)
    # Update values to the sheet
    new_sheet.insert_rows(updated_df.values.tolist())
    # add headers
    header = list(updated_df.columns)
    index = 1
    new_sheet.insert_row(header, index)

In [46]:
file_name = "Pubs Contain Non-medical Keywords"
# auto_accept_final.to_csv(f'{file_name}.csv', index = True)
score.fillna(value="",inplace=True)
upload_data_to_gs(file_name, #You can name the tab as wangfang pubs
                     score.astype(str),    # the data you want to upload
                     raw_df_sheet)